### Riddler Classic:


TODO: Add dev process

This notebook is going to iterate through each possible start word and determine how often we get a correct guess in 3 or less turns

In [1]:
import pandas as pd
import random
from collections import defaultdict
from functions import excludeChars, includeChars, properPos, improperPos, charRemoveWords, idxRemoveWords, nextGuess
import time 

# sample process:
win_tracker = {}

# read in myster words
mystery_corpus = pd.read_csv("data/mystery_words.csv", header = None)
mystery_list = [w[0] for w in mystery_corpus.values]
mystery_words = set(mystery_list)

# read in eligible guess words
guess_corpus = pd.read_csv("data/guess_words.csv", header = None)
guess_list = [w[0] for w in guess_corpus.values]

start = time.time()
for starter in guess_list[:10]:
    wins = 0
    print(f"Testing guess word: {starter}")
    for mystery_word in mystery_words:
        exclude_chars = set() # words can't include these 
        include_chars = set() # words need to include these
        proper_pos = defaultdict(set) # dictionary that stores the proper indices for a letter...note it needs to be val of list for multiple letters
        improper_pos = defaultdict(set) # dictionary that stores the improper indices for a letter...note it needs to be val of list for multiple letters
        guess = starter # our starter is our first guess
        guess_words = set(guess_list.copy())
        
        i = 0
        
        while i < 3:
            # check if we guesses correct:
            #print(starter, guess)
            if guess == mystery_word:
                wins += 1
                break


            # not accepted:
            exclude_chars.update(excludeChars(guess, mystery_word))

            # accepted:
            include_chars.update(includeChars(guess, mystery_word))

            #### We can now think through positions: 

            # proper pos
            temp_dict = properPos(guess,mystery_word)
            for k, v in temp_dict.items():
                proper_pos[k] = proper_pos[k].union(v)

            # ugly cleanup step - TODO: Fix
            new_dict = defaultdict(set)
            for k, v in proper_pos.items():
                if len(proper_pos[k]) != 0:
                    new_dict[k] = v
            proper_pos = new_dict

            # improper pos
            temp_dict = improperPos(guess,mystery_word, proper_pos)
            for k, v in temp_dict.items():
                improper_pos[k] = improper_pos[k].union(v)

            # ugly cleanup step - TODO: Fix
            new_dict = defaultdict(set)
            for k, v in improper_pos.items():
                if len(improper_pos[k]) != 0:
                    new_dict[k] = v
            improper_pos = new_dict

            # Move into reduce step 
            # we first reduce our set of words down based on excluded & included chars 
            new_words = charRemoveWords(exclude_chars, include_chars, guess_words)

            # we then reduce our set of words down based on proper & improper infices
            guess_words = idxRemoveWords(proper_pos, improper_pos, new_words)

            ### Make another guess 
            guess = nextGuess(include_chars, guess_words)
            i += 1

    # add wins for starter
    win_tracker[starter] = wins
    
print(f"Total time: {time.time() - start:.3f}")
win_p = win_tracker[max(win_tracker, key=win_tracker.get)] / len(guess_list)
print(f"Win prob is: {win_p:.5f}")

Testing guess word: aahed
Testing guess word: aalii
Testing guess word: aargh
Testing guess word: aarti
Testing guess word: abaca
Testing guess word: abaci
Testing guess word: aback
Testing guess word: abacs
Testing guess word: abaft
Testing guess word: abaka
Total time: 262.890
Win prob is: 0.01257


In [ ]:
# Note: This is very slow - need to refactor with Numpy. 